In [ ]:
# default_exp data.DeblurGANv2

In [ ]:
# export
import os
import cv2
import random
import numpy as np
import torch
import torch.utils.data as data
from skimage.transform import pyramid_gaussian
from defocus.data.common import *

In [ ]:
# export
'''
Here is a choice: do I switch to albumentations like Kupyn or 
keep Seungjun's basic augmentations?
Also maybe kornia?
'''
class Dataset(data.Dataset):
    # Seungjun's defaults for augmentations
    # https://github.com/SeungjunNah/DeepDeblur-PyTorch/blob/master/src/data/common.py
    
    # note that this dataset's only difference is that it does not output a pyramid but
    # single images.
    def __init__(self, root_folder, image_pair_list,
                 mode='train', 
                 augmentations={'hflip':0.5, 
                                'vflip':0.0, 
                                'rot90':0.5, 
                                'channel_shuffle':True, 
                                'saturation':True},
                 crop_size=256,
                 pyramid_scales=3,
                 ):
        super(Dataset, self).__init__()
        
        self.mode = mode
        self.augmentations = augmentations
        self.crop_size = 256
        self.pyramid_scales = pyramid_scales
        input_list = []
        target_list = []
        with open(image_pair_list, 'r') as f:
            for line in f:
                input_filename = line.split(' ')[0].strip('\n').strip('\t')
                target_filename = line.split(' ')[1].strip('\n').strip('\t')
                input_filepath = os.path.join(root_folder, input_filename)
                target_filepath = os.path.join(root_folder, target_filename)
                input_list.append(input_filepath)
                target_list.append(target_filepath)
        self.input_list = input_list
        self.target_list = target_list
        assert len(self.input_list) == len(self.target_list)
        self.length = len(self.input_list)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        # remember that opencv has BGR order so read and convert
        input_ = cv2.imread(self.input_list[idx])[:,:,::-1]
        target = cv2.imread(self.target_list[idx])[:,:,::-1]
        
        # if testing, do nothing
        if self.mode=='test' or self.mode=='validation':
            pass
        else:
            H, W, C = input_.shape
            patch_y = random.randrange(0, H-self.crop_size+1)
            patch_x = random.randrange(0, W-self.crop_size+1)
            input_ = crop(input_, patch_x, patch_y)
            target = crop(target, patch_x, patch_y)
        
            # write all augmentations explicitly. why not?
            if random.random() < self.augmentations['hflip']:
                input_ = hflip(input_)
                target = hflip(target)
            if random.random() < self.augmentations['vflip']:
                input_ = vflip(input_)
                target = vflip(target)
            if random.random() < self.augmentations['rot90']:
                # clockwise/counter-clockwise
                if random.random() < 0.5:
                    input_ = vflip(input_)
                    target = vflip(target)
                input_ = rot90(input_)
                target = rot90(target)
            if self.augmentations['channel_shuffle']:
                # note that this is actually a lower probability
                # e.g. shuffled order may just be [0,1,2] again
                rgb_order = [0,1,2]
                random.shuffle(rgb_order)
                input_ = channel_shuffle(input_, rgb_order)
                target = channel_shuffle(target, rgb_order)            
            if self.augmentations['saturation']:
                # Seungjun's defaults
                modifier = random.uniform(0.5, 1.5)
                input_ = saturation(input_, modifier)
                target = saturation(target, modifier)
            # noise only for the input
            # remember the GAN diffaug thing, may help if noise addition is non-leaky?
            input_ = add_gaussian_noise(input_)
            
        input_ = torch.from_numpy(np.ascontiguousarray(input_.astype(np.float32).transpose(2, 0, 1)))
        target = torch.from_numpy(np.ascontiguousarray(target.astype(np.float32).transpose(2, 0, 1)))
        
        return input_, target

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_model.ipynb.
Converted 02_architecture_common.ipynb.
Converted 03_architecture_MSResNet.ipynb.
Converted 04_dataset_common.ipynb.
Converted 05_dataset_MSResNet.ipynb.
Converted 06_trainer_MSResNet.ipynb.
Converted 07_metrics.ipynb.
Converted 08_architecture_DeblurGANv2.ipynb.
Converted 09_dataset_DeblurGANv2.ipynb.
Converted 99_diffaugment.ipynb.
Converted Tutorial_without_lightning.ipynb.
Converted model_without_lightning.ipynb.
Converted trials.ipynb.
